In [ ]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from collections import Counter

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score, auc
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [ ]:
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import Conv2D, Activation, Dense, Lambda, Input, MaxPooling2D, Dropout, Flatten, Reshape, UpSampling2D, Concatenate
from keras.losses import mse, binary_crossentropy
from keras import backend as K
from keras.utils import plot_model, to_categorical
from keras.callbacks import EarlyStopping


import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
import random
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
from itertools import chain


In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape((-1, 784)).astype('float32') / 255.0
x_test = x_test.reshape((-1, 784)).astype('float32') / 255.0
# y_train = to_categorical(y_train, 10)
# y_test = to_categorical(y_test, 10)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)
mnist_digits = np.concatenate([x_train, x_test], axis=0)
print(mnist_digits.shape)

# New training and testing data with only 0,3,1,9

In [ ]:
indices_to_pick_train = []
indices_to_pick_test = []
for cluster_index in [0,3,1,9]:
    print("checking {}".format(cluster_index))
    C_i_train = np.where(y_train == cluster_index)[0].tolist()
    indices_to_pick_train.append(C_i_train)
    
    C_i_test = np.where(y_test == cluster_index)[0].tolist()
    indices_to_pick_test.append(C_i_test)
    

In [ ]:
indices_to_pick_train = list(chain.from_iterable(indices_to_pick_train)) # flatten the 2D list
indices_to_pick_test = list(chain.from_iterable(indices_to_pick_test)) # flatten the 2D list

In [ ]:
print(len(indices_to_pick_train))
print(len(indices_to_pick_test))

In [ ]:
train_data = []
train_labels = []
test_data = []
test_labels = []

for index in indices_to_pick_train:
    train_data.append(x_train[index])
    train_labels.append(y_train[index]) 

In [ ]:
for index in indices_to_pick_test:
    test_data.append(x_test[index])
    test_labels.append(y_test[index])

In [ ]:
X_train = np.array(train_data)
Y_train = np.array(train_labels)
print(X_train.shape, Y_train.shape)

In [ ]:
X_test = np.array(test_data)
Y_test = np.array(test_labels)
print(X_test.shape, Y_test.shape)

In [ ]:
np.save('/home/dsarkar/compute_canada/MNIST/dataset/MNIST4_new_train_x.npy',X_train)
np.save('/home/dsarkar/compute_canada/MNIST/dataset/MNIST4_new_train_y.npy',Y_train)
np.save('/home/dsarkar/compute_canada/MNIST/dataset/MNIST4_new_test_x.npy',X_test)
np.save('/home/dsarkar/compute_canada/MNIST/dataset/MNIST4_new_test_y.npy',Y_test)

In [ ]:
X_train = np.load('/home/dsarkar/compute_canada/MNIST/dataset/MNIST4_new_train_x.npy')
Y_train = np.load('/home/dsarkar/compute_canada/MNIST/dataset/MNIST4_new_train_y.npy')
X_test = np.load('/home/dsarkar/compute_canada/MNIST/dataset/MNIST4_new_test_x.npy')
Y_test = np.load('/home/dsarkar/compute_canada/MNIST/dataset/MNIST4_new_test_y.npy')
print(X_train.shape, Y_train.shape);print(X_test.shape, Y_test.shape)

# VAE

In [ ]:
# reparameterization trick
# instead of sampling from Q(z|X), sample eps = N(0,I)
# z = z_mean + sqrt(var)*eps
def sampling(args):
    """Reparameterization trick by sampling fr an isotropic unit Gaussian.
    # Arguments:
        args (tensor): mean and log of variance of Q(z|X)
    # Returns:
        z (tensor): sampled latent vector
    """
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    # by default, random_normal has mean=0 and std=1.0
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

In [ ]:
image_shape = (28, 28, 1)
original_dim = image_shape[0] * image_shape[1]
input_shape = (original_dim,)
batch_size = 64
latent_dim = 5
epochs = 30

# encoder
inputs = Input(shape=input_shape)
x = Reshape(image_shape)(inputs)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2))(x)
x = Dropout(0.25)(x)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
z_mean = Dense(latent_dim, name='z_mean')(x)
z_log_var = Dense(latent_dim, name='z_log_var')(x)
z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])

encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')
encoder.summary()
#plot_model(encoder, to_file='vae_cnn_encoder.png', show_shapes=True)

# decoder
latent_inputs = Input(shape=(latent_dim,), name='z_sampling')
#label_inputs = Input(shape=(10,), name='label')
#x = Concatenate()([latent_inputs, label_inputs])
x = Dense(128, activation='relu')(latent_inputs)
x = Dense(14 * 14 * 64, activation='relu')(x)
x = Reshape((14, 14, 64))(x)
x = Dropout(0.25)(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)
outputs = Reshape(input_shape)(x)

decoder = Model([latent_inputs], outputs, name='decoder')
decoder.summary()
plot_model(decoder, to_file='vae_cnn_decoder.png', show_shapes=True)

# variational autoencoder
outputs = decoder([encoder(inputs)[2]])

vae = Model([inputs], outputs, name='vae_mlp')
vae.summary()
#plot_model(vae, to_file='vae_cnn.png', show_shapes=True)

In [ ]:
reconstruction_loss = binary_crossentropy(inputs, outputs)
reconstruction_loss *= original_dim
kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
kl_loss = K.sum(kl_loss, axis=-1)
kl_loss *= -0.5
#vae_loss = K.mean(reconstruction_loss + kl_loss)
vae_loss = reconstruction_loss + kl_loss
vae.add_loss(vae_loss)
vae.compile(optimizer='Adam')

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

In [ ]:
import os

weights_file = '/home/dsarkar/compute_canada/MNIST/model/vae_cnn_mnist4_z5.h5'

if os.path.exists(weights_file):
    vae.load_weights(weights_file)
    print('Loaded weights!')
else:
    history = vae.fit(X_train,
            epochs=epochs,
            batch_size=batch_size, 
           validation_split = 0.2,
            #validation_data=(X_test, Y_test),
             verbose=1, callbacks=[es])
    vae.save_weights('/home/dsarkar/compute_canada/MNIST/model/vae_cnn_mnist4_z5.h5')

In [ ]:
# summarize history for loss
plt.figure(figsize=(10,10))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss for latent dim = 2')
plt.ylabel('MSE value')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()

In [ ]:
latent_space_train = encoder.predict(X_train)[2]
print(latent_space_train.shape)

latent_space_test = encoder.predict(X_test)[2]
latent_space_test.shape

In [ ]:
latent_space_train = encoder.predict(X_train)[2]
print(latent_space_train.shape)
np.save('/home/dsarkar/compute_canada/MNIST/dataset/MNIST4_train_latent_space_z5.npy',latent_space_train)

In [ ]:
latent_space_test = encoder.predict(X_test)[2]
np.save('/home/dsarkar/compute_canada/MNIST/dataset/MNIST4_test_latent_space_z5.npy',latent_space_test)
latent_space_test.shape

In [ ]:
latent_space_train = np.load('/home/dsarkar/compute_canada/MNIST/dataset/MNIST4_train_latent_space_z5.npy')
print(latent_space_train.shape, Y_train.shape)

In [ ]:
latent_space_test = np.load('/home/dsarkar/compute_canada/MNIST/dataset/MNIST4_test_latent_space_z5.npy')
print(latent_space_test.shape,Y_test.shape)

# NMI

In [ ]:
from sklearn.metrics.cluster import normalized_mutual_info_score
kmeans = KMeans(n_clusters=4, random_state=0).fit(latent_space_train)
kmeans.labels_

In [ ]:
normalized_mutual_info_score(kmeans.labels_,Y_train)



In [ ]:
kmeans = KMeans(n_clusters=4, random_state=0).fit(X_train)
normalized_mutual_info_score(kmeans.labels_,Y_train)


# Random sampling on the latent space

In [ ]:
tuned_parameters = [{'kernel': ['rbf'], 'C' : [0.1, 1, 10, 100]},
                    {'kernel': ['linear'], 'C' : [0.1, 1, 10, 100]}]

# ,
#                    {'kernel': ['poly'], 'C' : [0.1, 1, 10, 100], 'gamma': [1,0.1,0.01,0.001]},
#                    {'kernel': ['sigmoid'], 'C' : [0.1, 1, 10, 100], 'gamma': [1,0.1,0.01,0.001]}]
score = 'accuracy'
clf = GridSearchCV(SVC(), tuned_parameters, scoring=score, n_jobs=-1,refit=True,verbose=1)

## On real data

In [ ]:
mean_random = []
sd_random = []

for coreset_size in range(20,501,20): # start from 20 labeled points
    print("*********************** Training on {} points ***********************".format(coreset_size))
    accuracy = [] # calculate accuracy of 100 iterations
    c = list(zip(X_train,Y_train))
    iterations = 0
    while iterations < 100: # run 100 simulations and take average 
        train_data = []
        train_labels = []
        for (data,label) in random.sample(c,coreset_size):
            train_data.append(data)
            train_labels.append(label)  
        train_x = np.array(train_data)
        train_y = np.array(train_labels)
        
        print()
        print("Distribution of data in the training points")
        print(Counter(train_y)) 

        clf.fit(train_x, train_y)
        print("Best parameters set found on {} data points:".format(coreset_size))
        print(clf.best_params_)
        print()
        y_true, y_pred = Y_test, clf.predict(X_test)
        accuracy.append(accuracy_score(y_true, y_pred))
        iterations += 1
    accuracy = np.asarray(accuracy)
    mean_accuracy = accuracy.mean()
    sd_accuracy = accuracy.std()
    
    mean_random.append(mean_accuracy)
    sd_random.append(sd_accuracy)

In [ ]:
mean_random = np.array(mean_random)
sd_random = np.array(sd_random)


np.save('/home/dsarkar/compute_canada/MNIST/result/mean_accuracy_rs_MNIST4_input_space.npy',mean_random)
np.save('/home/dsarkar/compute_canada/MNIST/result/sd_accuracy_rs_MNIST4_input_space.npy',sd_random)

In [ ]:
mean_random

## On latent space

In [ ]:
mean_random = []
sd_random = []

for coreset_size in range(20,501,20): # start from 20 labeled points
    print("*********************** Training on {} points ***********************".format(coreset_size))
    accuracy = [] # calculate accuracy of 100 iterations
    c = list(zip(latent_space_train,Y_train))
    iterations = 0
    while iterations < 100: # run 100 simulations and take average 
        train_data = []
        train_labels = []
        for (data,label) in random.sample(c,coreset_size):
            train_data.append(data)
            train_labels.append(label)  
        train_x = np.array(train_data)
        train_y = np.array(train_labels)
        
        print()
        print("Distribution of data in the training points")
        print(Counter(train_y)) 

        clf.fit(train_x, train_y)
        print("Best parameters set found on {} data points:".format(coreset_size))
        print(clf.best_params_)
        print()
        y_true, y_pred = Y_test, clf.predict(latent_space_test)
        accuracy.append(accuracy_score(y_true, y_pred))
        iterations += 1
    accuracy = np.asarray(accuracy)
    mean_accuracy = accuracy.mean()
    sd_accuracy = accuracy.std()
    
    mean_random.append(mean_accuracy)
    sd_random.append(sd_accuracy)

In [ ]:
mean_random = np.array(mean_random)
sd_random = np.array(sd_random)


np.save('/home/dsarkar/compute_canada/MNIST/result/mean_accuracy_rs_MNIST4_z5.npy',mean_random)
np.save('/home/dsarkar/compute_canada/MNIST/result/sd_accuracy_rs_MNIST4_z5.npy',sd_random)

In [ ]:
mean_random

# K-means coreset on latent space

In [ ]:
mean_accuracy_coreset 

# K=40

In [ ]:
kmeans = KMeans(n_clusters=40, random_state=0).fit(latent_space_train)
np.unique(kmeans.labels_)

In [ ]:
mean_accuracy_coreset = []
sd_accuracy_coreset = []

for coreset_size in range(40,501,40): # start from 60 labeled points
    print("*********************** Training on {} points ***********************".format(coreset_size))

    accuracy = []
    m = int(coreset_size/40) # m=B/K, number of points from each cluster
    iterations = 0
    while iterations < 100: # run 100 simulations and take average 
        train_data = []
        train_labels = []
        indices_to_pick = []
        
        print("Choosing {} points from each cluster".format(m))
        for cluster_index in range(40):
            C_i = np.where(kmeans.labels_ == cluster_index)[0].tolist()
            sample_i = random.sample(C_i, m)
            indices_to_pick.append(sample_i)
        
        indices_to_pick = list(chain.from_iterable(indices_to_pick)) # flatten the 2D list
        
        assert len(indices_to_pick)==coreset_size, "Sample size mismatch!!!!"
        
        for index in indices_to_pick:
            train_data.append(latent_space_train[index])
            train_labels.append(Y_train[index]) 
        
        train_x = np.array(train_data)
        train_y = np.array(train_labels)
        
        print()
        print("Distribution of data in the training points")
        print(Counter(train_y))

        clf.fit(train_x, train_y)
        print("Best parameters set found on {} data points:".format(coreset_size))
        print(clf.best_params_)
        print()
        y_true, y_pred = Y_test, clf.predict(latent_space_test)
        accuracy.append(accuracy_score(y_true, y_pred))
        iterations += 1

    accuracy = np.asarray(accuracy)
    mean_accuracy = accuracy.mean()
    sd_accuracy = accuracy.std()


    mean_accuracy_coreset.append(mean_accuracy)
    sd_accuracy_coreset.append(sd_accuracy)

In [ ]:
mean_accuracy_coreset = np.array(mean_accuracy_coreset)
sd_accuracy_coreset = np.array(sd_accuracy_coreset)

np.save('/home/dsarkar/compute_canada/MNIST/result/mean_accuracy_cs_MNIST4_z5_K40.npy',mean_accuracy_coreset)
np.save('/home/dsarkar/compute_canada/MNIST/result/sd_accuracy_cs_MNIST4_z5_K40.npy',sd_accuracy_coreset)

In [ ]:
mean_accuracy_coreset #after 100 iterations each

### On original data

 # K-means coreset on input space

## K=40

In [ ]:
kmeans = KMeans(n_clusters=40, random_state=0).fit(X_train)

In [ ]:
mean_accuracy_coreset = []
sd_accuracy_coreset = []

for coreset_size in range(40,501,40): # start from 60 labeled points
    print("*********************** Training on {} points ***********************".format(coreset_size))

    accuracy = []
    m = int(coreset_size/40) # m=B/K, number of points from each cluster
    iterations = 0
    while iterations < 100: # run 100 simulations and take average 
        train_data = []
        train_labels = []
        indices_to_pick = []
        
        print("Choosing {} points from each cluster".format(m))
        for cluster_index in range(40):
            C_i = np.where(kmeans.labels_ == cluster_index)[0].tolist()
            sample_i = random.sample(C_i, m)
            indices_to_pick.append(sample_i)
        
        indices_to_pick = list(chain.from_iterable(indices_to_pick)) # flatten the 2D list
        
        assert len(indices_to_pick)==coreset_size, "Sample size mismatch!!!!"
        
        for index in indices_to_pick:
            train_data.append(X_train[index])
            train_labels.append(Y_train[index]) 
        
        train_x = np.array(train_data)
        train_y = np.array(train_labels)
        
        print()
        print("Distribution of data in the training points")
        print(Counter(train_y))

        clf.fit(train_x, train_y)
        print("Best parameters set found on {} data points:".format(coreset_size))
        print(clf.best_params_)
        print()
        y_true, y_pred = Y_test, clf.predict(X_test)
        accuracy.append(accuracy_score(y_true, y_pred))
        iterations += 1

    accuracy = np.asarray(accuracy)
    mean_accuracy = accuracy.mean()
    sd_accuracy = accuracy.std()


    mean_accuracy_coreset.append(mean_accuracy)
    sd_accuracy_coreset.append(sd_accuracy)

In [ ]:
mean_accuracy_coreset = np.array(mean_accuracy_coreset)
sd_accuracy_coreset = np.array(sd_accuracy_coreset)

np.save('/home/dsarkar/compute_canada/MNIST/result/mean_accuracy_cs_MNIST4_input_space_K40.npy',mean_accuracy_coreset)
np.save('/home/dsarkar/compute_canada/MNIST/result/sd_accuracy_cs_MNIST4_input_space_K40.npy',sd_accuracy_coreset)

In [ ]:
mean_accuracy_coreset